In [2]:
import pandas as pd
import folium

In [65]:
import folium
from folium.plugins import HeatMap

In [136]:
df = pd.read_csv('data_with_coords.csv')
df.head(5)

,Unnamed: 0,addr_street,addr_number,addr_building,addr_letter,addr_district,comm_type,comm_num,comm_room_num,data_series,...,param_failure,repair_year,repair_job,rfc_shaftcount,roof_metalarea,seng_liftcount,seng_pzu,special_basementarea,lat,lng
0,0,Королева пр.,31,1,В,Приморский,3 комн.,1,3,NaN,...,0.0,2014,Капитальный ремонт лифтов с частичной заменой ...,4.0,NaN,4.0,NaN,916.9,59.940737,30.291174
1,1,Дачный пр.,33,1,А,Кировский,"2 комн., 3 комн.","2, 2","4, 6",1-528 КП- 42,...,0.0,2002; 2004; 2004; 2004; 2005; 2007; 2008; 2009...,Благоустройство придомовой территории; Благоус...,5.0,NaN,5.0,5.0,1444.0,59.944576,30.268982
2,2,Будапештская ул.,34,NaN,А,Фрунзенский,3 комн.,2,6,1-528-КП-40,...,0.0,2005; 2005; 2010; 2012; 2014,Капитальный ремонт лифтов с полной заменой обо...,1.0,NaN,1.0,1.0,353.0,59.940908,30.264523
3,3,Бухарестская ул.,86,2,А,Фрунзенский,"2 комн., 3 комн.","28, 4","56, 12",1ЛГ-502-9,...,0.0,NaN,NaN,NaN,NaN,NaN,9.0,1373.8,59.944815,30.272974
4,4,"г.Пушкин, Красносельское ш.",65,NaN,А,Пушкинский,3 комн.,3,9,528,...,0.0,2007; 2007; 2008,Прочие работы по благоустройству; Ремонт и вос...,NaN,NaN,NaN,7.0,1282.0,59.935427,30.276184


#### Коммунальное жилье (плотность??? как правильно назвать то, что на хитмапе)

Для каждого дома найдем суммарное количество коммунальных КОМНАТ. Представим в виде heatmap.

In [131]:
fnct = lambda x: sum([int(xx) for xx in x.split(', ')])
df['comm_room_sum'] = df['comm_room_num'].fillna(0).astype('str').map(fnct)
df['comm_intensity'] = df['comm_room_sum']/df['comm_room_sum'].max()
df['comm_room_sum'].quantile(q=0.95)

70.0

In [135]:
m = folium.Map(location=[59.9393773,30.3181802], zoom_start=10)
HeatMap(data=df.dropna(subset=('comm_num',))[['lat', 'lng', 'comm_room_sum']].values.tolist(),
        max_val=70.0,
        radius=8, max_zoom=13).add_to(m)
m

#### Годы постройки

Сохранившиеся жилые многоквартирные здания дореволюционной постройки

In [146]:
data_columns = ['data_buildingdate',]
num_df = (df.drop(data_columns, axis=1).join(df[data_columns].apply(pd.to_numeric, errors='coerce')))

In [147]:
num_df = num_df.dropna(subset=('data_buildingdate',))

In [148]:
m1 = folium.Map(location=[59.9393773,30.3181802], zoom_start=10)
HeatMap(data=num_df[num_df['data_buildingdate'] < 1917][['lat', 'lng']].values.tolist(),
        radius=8, max_zoom=13).add_to(m1)
m1